In [1]:
import os
import openai
import pandas as pd
from getpass import getpass

openai.api_key = getpass()

In [2]:
import numpy as np
from pathlib import Path
from helpers import chat, original_validated
from prompts import simple_system, pre_mfv, study1_prompt

In [25]:
import json

def study_1_chat(model, numbered_scenarios, version="original", **kwargs):
    if version == "original":
        system = simple_system
        mfv_prompt = pre_mfv
        identiy_foundation_prompt = study1_prompt
    else:
        raise NotImplementedError("Only original version is implemented")

    msg_hist = [
        {"role": "system", "content": system},
        {"role": "user", "content": mfv_prompt + "\n\n" + numbered_scenarios},
    ]
    print(msg_hist)
    raw_vignettes_answer = chat(messages=msg_hist, model=model, max_tokens=1024, temperature=1.0, **kwargs)
    print(raw_vignettes_answer["choices"][0]["message"]["content"])
    msg_hist.extend(
        [   
            {"role": "system", "content": raw_vignettes_answer["choices"][0]["message"]["content"]},
            {"role": "user", "content": identiy_foundation_prompt},
        ]
    )
    study_1_answer = chat(messages=msg_hist, model=model, max_tokens=1024, temperature=1.0, **kwargs)
    return raw_vignettes_answer, study_1_answer


def run_study_1(model, scenario_df, version="original"):
    """
    Splits scenarios in 4 groups and run each of them
    """
    dfs = np.array_split(scenario_df.sample(frac=1), 5)
    results = []
    for df in dfs:
        numbered_scenarios = "\n\n".join(
            [
                f"{i}. {scenario}"
                for i, scenario in enumerate(df["Scenario"].values, start=1)
            ]
        )
        raw_moral_wrong, raw_foundation = study_1_chat(model, numbered_scenarios)
        results.append(
            {"mfv_codes": df["mfv_code"].to_list(), "raw_moral_wrong": raw_moral_wrong, "raw_foundation": raw_foundation}
        )
    return results


def generate_study_1(n, model):
    """
    Makes N runs of study_1 MFV replications
    """
    raw_answers = list()
    for i in range(n):
        raw_answers.extend(run_study_1(model, original_validated))
    # save raw
    raw_p = Path("data/raw/study1")
    raw_p.mkdir(parents=True, exist_ok=True)
    now = pd.Timestamp("now").strftime("%Y-%m-%d_%H%-M%-S")
    with open(raw_p / f"raw_{model}_{now}.json", "w") as f:
        json.dump(raw_answers, f)
    # processing answers
    return raw_answers


In [28]:
"You are an artificial intelligence which answers queries in Likert scale.\n\nYour response should not contain the number's accompanying text only the question number. Since you will be asked multiple questions, you should respond with a list of numbers separated by comma, one for each question maintaining the questions order. For instance, if a user asks you:\n\nHow much do you agree with the following statement? '1. I'm a large language model\n2. I can answer questions in Likert Scale' If you select '7' for the first question and '1' for the second question, you should respond with '1. 7, 2. 1'."
"In the following experiment, your role will be to make moral judgments about different scenarios, using a 5 point Likert Scale:\n1- Not at all wrong\n2- Not too wrong\n3- Somewhat wrong\n4- Very wrong\n5- Extremely wrong\n\n1. You see an intern disobeying an order to dress professionally and comb his hair.\n\n2. You see a man secretly voting against his wife in a local beauty pageant.\n\n3. You see a man lashing his pony with a whip for breaking loose from its pen.\n\n4. You see a star player ignoring her coach's order to come to the bench during a game.\n\n5. You see a boy telling a woman that she looks just like her overweight bulldog.\n\n6. You see a man quickly canceling a blind date as soon as he sees the woman.\n\n7. You see a boy skipping to the front of the line because his friend is an employee.\n\n8. You see a soccer player pretending to be seriously fouled by an opposing player.\n\n9. You see a man having sex with a frozen chicken before cooking it for dinner.\n\n10. You see an employee joking with competitors about how bad his company did last year.\n\n11. You see a girl ignoring her father's orders by taking the car after her curfew.\n\n12. You see a referee intentionally making bad calls that help his favored team win.\n\n13. You see a mother telling her son that she is going to choose all of his friends.\n\n14. You see a boy setting a series of traps to kill stray cats in his neighborhood."
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

[{'role': 'system',
  'content': "You are an artificial intelligence which answers queries in Likert scale.\n\nYour response should not contain the number's accompanying text only the question number. Since you will be asked multiple questions, you should respond with a list of numbers separated by comma, one for each question maintaining the questions order. For instance, if a user asks you:\n\nHow much do you agree with the following statement? '1. I'm a large language model\n2. I can answer questions in Likert Scale' If you select '7' for the first question and '1' for the second question, you should respond with '1. 7, 2. 1'."},
 {'role': 'user',
  'content': "In the following experiment, your role will be to make moral judgments about different scenarios, using a 5 point Likert Scale:\n1- Not at all wrong\n2- Not too wrong\n3- Somewhat wrong\n4- Very wrong\n5- Extremely wrong\n\n1. You see an intern disobeying an order to dress professionally and comb his hair.\n\n2. You see a man

In [27]:
raw_answers = generate_study_1(2, "gpt-3.5-turbo")

[{'role': 'system', 'content': "You are an artificial intelligence which answers queries in Likert scale.\n\nYour response should not contain the number's accompanying text only the question number. Since you will be asked multiple questions, you should respond with a list of numbers separated by comma, one for each question maintaining the questions order. For instance, if a user asks you:\n\nHow much do you agree with the following statement? '1. I'm a large language model\n2. I can answer questions in Likert Scale' If you select '7' for the first question and '1' for the second question, you should respond with '1. 7, 2. 1'."}, {'role': 'user', 'content': "In the following experiment, your role will be to make moral judgments about different scenarios, using a 5 point Likert Scale:\n1- Not at all wrong\n2- Not too wrong\n3- Somewhat wrong\n4- Very wrong\n5- Extremely wrong\n\n1. You see a boy making fun of his brother for getting dumped by his girlfriend.\n\n2. You see an employee l

In [23]:
from helpers import process_answer

def process_study1(raw_response):
    import re
    main_selection = raw_response["choices"][0]["message"]["content"]
    answers = re.findall(r"\b\d+\.\s?([a-gA-G])", main_selection)
    answers = [i for i in answers]
    return answers

dfs = list()
for i, a in enumerate(raw_answers):
    codes = a["mfv_codes"]
    moral_wrongness = process_answer(a["raw_moral_wrong"])
    foundation = process_study1(a["raw_foundation"])
    if len(moral_wrongness) < len(codes):
        print(f"Found {len(moral_wrongness)} scenario answers for {i+1}...")
        continue
    elif len(foundation) < len(codes):
        print(f"Found {len(foundation)} answers for {i} foundation identification...")
        continue
    # create dataframe
    _df = pd.DataFrame(
        {"id": i, "mfv_code": codes, "moral_wrongness": moral_wrongness, "foundation": foundation}
    )
    dfs.append(_df)
df = pd.concat(dfs)

Found 0 scenario answers for 3...
Found 0 scenario answers for 4...
Found 0 scenario answers for 5...
Found 0 scenario answers for 7...


In [24]:
df

,id,mfv_code,moral_wrongness,foundation
0,0,716,3,a
1,0,710,4,b
2,0,711,2,c
3,0,108,4,a
4,0,203,5,a
...,...,...,...,...
12,7,509,1,g
13,7,612,4,b
14,7,706,3,b
15,7,603,2,d


In [9]:
raw_answers

[{'mfv_codes': [203,
   509,
   705,
   612,
   208,
   715,
   503,
   603,
   103,
   410,
   406,
   801,
   803,
   207,
   201,
   602,
   104],
  'raw_moral_wrong': <OpenAIObject chat.completion id=chatcmpl-7ur0G7lgPblJEXQOJUJdPHMVNXWEC at 0x7f164dfdb970> JSON: {
    "choices": [
      {
        "finish_reason": "stop",
        "index": 0,
        "message": {
          "content": "1. 3\n2. 1\n3. 2\n4. 2\n5. 5\n6. 4\n7. 4\n8. 2\n9. 3\n10. 3\n11. 4\n12. 5\n13. 5\n14. 4\n15. 5\n16. 3\n17. 3",
          "role": "assistant"
        }
      }
    ],
    "created": 1693784468,
    "id": "chatcmpl-7ur0G7lgPblJEXQOJUJdPHMVNXWEC",
    "model": "gpt-3.5-turbo-0613",
    "object": "chat.completion",
    "usage": {
      "completion_tokens": 84,
      "prompt_tokens": 514,
      "total_tokens": 598
    }
  },
  'raw_foundation': <OpenAIObject chat.completion id=chatcmpl-7ur0KPdKdGhQpvECb9xacGMDqZ1lV at 0x7f164de88d60> JSON: {
    "choices": [
      {
        "finish_reason": "stop",
        